In [4]:
# 📊 Data Handling
import pandas as pd
import numpy as np

# 📈 Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# ⚙️ Preprocessing & Scaling
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# 📈 Analysis
import scipy.stats as stats
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
  
# 🧠 Modeling
from sklearn.cluster import KMeans, DBSCAN
import hdbscan
from sklearn.metrics import silhouette_score

 # Part 1: Basic Information

SQL Query:
<br>
-- Step 1: همه آدرس‌های قرارداد
WITH contract_addresses AS (
  SELECT DISTINCT to_address AS address
  FROM `bigquery-public-data.crypto_ethereum.traces`
  WHERE call_type = 'create' AND to_address IS NOT NULL
),

<br> -- Step 2: آدرس‌هایی که تراکنش داشتند و تاریخچه‌شون
active_wallets AS (
  SELECT
    from_address AS address,
    COUNT(*) AS tx_count,
    MIN(block_timestamp) AS first_tx,
    MAX(block_timestamp) AS last_tx
  FROM `bigquery-public-data.crypto_ethereum.transactions`
  WHERE block_timestamp >= TIMESTAMP("2025-04-01")
  GROUP BY from_address
)

<br> -- Step 3: فقط EOAها رو نگه می‌داریم و به موجودی وصله می‌زنیم
SELECT
  a.address,
  a.tx_count,
  a.first_tx,
  a.last_tx,
  b.eth_balance
FROM
  active_wallets a
LEFT JOIN
  contract_addresses c
ON
  a.address = c.address
JOIN
  `bigquery-public-data.crypto_ethereum.balances` b
ON
  a.address = b.address
WHERE
  c.address IS NULL -- فقط EOAها
ORDER BY
  tx_count DESC;

In [5]:
df1 = pd.read_csv('/Users/bahareh/Desktop/Data/part 1.csv')

df1['first_tx'] = pd.to_datetime(df1['first_tx'])
df1['last_tx'] = pd.to_datetime(df1['last_tx'])

df1['eth_balance'] = pd.to_numeric(df1['eth_balance'], errors='coerce')
df1['eth'] =round(df1['eth_balance'] / 1e18)
df1 = df1.rename(columns={'address': 'wallet_address'})

df1 = df1.drop(['eth_balance'],axis=1)
df1.head()

,wallet_address,tx_count,first_tx,last_tx,eth
0,0x1a1c87d9a6f55d3bbb064bff1059ad37b6bdc097,580800,2025-04-01 00:00:59+00:00,2025-05-03 17:40:11+00:00,2.0
1,0x974caa59e49682cda0ad2bbe82983419a2ecc400,312795,2025-04-01 00:00:11+00:00,2025-05-03 17:46:35+00:00,10204.0
2,0x46340b20830761efd32832a74d7169b29feb9758,302996,2025-04-01 00:00:35+00:00,2025-05-03 17:46:11+00:00,19402.0
3,0xf70da97812cb96acdf810712aa562db8dfa3dbef,275100,2025-04-01 00:00:11+00:00,2025-05-03 17:46:35+00:00,368.0
4,0x264bd8291fae1d75db2c5f573b07faa6715997b5,270429,2025-04-01 00:00:11+00:00,2025-05-03 17:46:35+00:00,3346.0


# Part 2: Transaction Behavior

WITH contracts AS (
  SELECT DISTINCT to_address AS address
  FROM `bigquery-public-data.crypto_ethereum.traces`
  WHERE call_type = 'create' AND to_address IS NOT NULL
),

txs AS (
  SELECT
    block_timestamp,
    from_address,
    to_address,
    value,
    gas,
    gas_price
  FROM
    `bigquery-public-data.crypto_ethereum.transactions`
  WHERE
    block_timestamp >= TIMESTAMP("2025-03-15")
)

-- فقط تراکنش‌هایی که فرستنده‌شون EOA باشه:
SELECT
  t.*
FROM
  txs t
LEFT JOIN
  contracts c
ON
  t.from_address = c.address
WHERE
  c.address IS NULL

In [6]:
df2 = pd.read_csv('/Users/bahareh/Desktop/Data/part 2.csv')
df2['block_timestamp'] = pd.to_datetime(df2['block_timestamp'])
df2.head()

,block_timestamp,from_address,to_address,value,gas,gas_price
0,2025-05-03 15:47:47+00:00,0x463e5b673d1029989c9b059d36393c539bef9094,0xfce76faf8b213496ca23cea1385cef6b4bd9e940,120083601538014700,21000,1905103281
1,2025-05-03 15:48:11+00:00,0x3ed461af3ea8ed17e7722754c6080d3f0d064bc8,0xdac17f958d2ee523a2206206994597c13d831ec7,0,63209,726666060
2,2025-05-03 15:48:59+00:00,0x7e8242cb6e74e9b82a564877a5fe49c47e3d882b,0x7e8242cb6e74e9b82a564877a5fe49c47e3d882b,0,21000,510000000
3,2025-05-03 15:49:11+00:00,0x3dbdc1c775220bddafd265c05d6e4be7bf4ac81f,0xa5c8b1c32d05df6f4e6267485452f784e8ae5007,0,86988,890363963
4,2025-05-03 15:49:11+00:00,0xc35fb86f962ea955751a793a007b5cdd44f798d7,0xdac17f958d2ee523a2206206994597c13d831ec7,0,65116,495663673


In [7]:
wallet_stats = []

for address, group in df2.groupby('from_address'):
    group = group.sort_values('block_timestamp')
    group['value'] = pd.to_numeric(group['value'], errors='coerce')
    
    total_tx = len(group)
    avg_value = round(group['value'].mean()/ 1e18)
    
    if total_tx > 1:
        time_diffs = group['block_timestamp'].diff().dt.total_seconds() / 86400 
        avg_time_gap = avg_time_gap = time_diffs.mean() if total_tx > 1 else 0
    else:
        avg_time_gap = None

    min_date = group['block_timestamp'].min()
    max_date = group['block_timestamp'].max()
    total_days = (max_date - min_date).days + 1 
    weeks = total_days / 7 if total_days > 0 else 1
    months = total_days / 30 if total_days > 0 else 1

    tx_per_day = total_tx / total_days if total_days > 0 else total_tx
    tx_per_week = total_tx / weeks
    tx_per_month = total_tx / months

    wallet_stats.append({
        'wallet_address': address,
        'total_tx': total_tx,
        'avg_tx_value': avg_value,
        'tx_per_day': tx_per_day,
        'tx_per_week': tx_per_week,
        'tx_per_month': tx_per_month,
        'avg_time_gap_days': avg_time_gap
    })

result_df = pd.DataFrame(wallet_stats)

eth_price_usd = 3500

result_df['avg_value_usd'] = result_df['avg_tx_value'] * eth_price_usd

result_df['avg_time_gap_days'] = result_df['avg_time_gap_days'].apply(
    lambda x: 0 if pd.isna(x) or x < 1 else int(x)
)

result_df

,wallet_address,total_tx,avg_tx_value,tx_per_day,tx_per_week,tx_per_month,avg_time_gap_days,avg_value_usd
0,0x00000000000030e5959659622cb7eb50aa20ee52,5,0,0.104167,0.729167,3.125000,11,0
1,0x00000000000067f5b5b06c66a9a8d4445ba98083,12,0,0.244898,1.714286,7.346939,4,0
2,0x000000000000c8dcb423f2095460086e0b381044,5,0,0.227273,1.590909,6.818182,5,0
3,0x00000000000124d994209fbb955e0217b5c2eca1,17,0,0.346939,2.428571,10.408163,3,0
4,0x00000000000947821264914ad2c75f871aa2d026,32,0,0.653061,4.571429,19.591837,1,0
...,...,...,...,...,...,...,...,...
36294,0xfff7265cbac1d028a0e4029ba311ebd4f79573d3,1,0,1.000000,7.000000,30.000000,0,0
36295,0xfff7953ed69781fb8040a37b63773a9349bbd711,1,0,1.000000,7.000000,30.000000,0,0
36296,0xfff874445cdbc96743924a7ccad12f5d7f332d77,1,0,1.000000,7.000000,30.000000,0,0
36297,0xfff9326b779fb266a3ce48322d389f94312a18f2,1,0,1.000000,7.000000,30.000000,0,0


# Final Data

In [8]:
df_final = pd.merge(df1,result_df, on='wallet_address',how='inner')
df_final

,wallet_address,tx_count,first_tx,last_tx,eth,total_tx,avg_tx_value,tx_per_day,tx_per_week,tx_per_month,avg_time_gap_days,avg_value_usd
0,0x1a1c87d9a6f55d3bbb064bff1059ad37b6bdc097,580800,2025-04-01 00:00:59+00:00,2025-05-03 17:40:11+00:00,2.0,393,0,8.020408,56.142857,240.612245,0,0
1,0x974caa59e49682cda0ad2bbe82983419a2ecc400,312795,2025-04-01 00:00:11+00:00,2025-05-03 17:46:35+00:00,10204.0,396,0,8.250000,57.750000,247.500000,0,0
2,0x46340b20830761efd32832a74d7169b29feb9758,302996,2025-04-01 00:00:35+00:00,2025-05-03 17:46:11+00:00,19402.0,253,1,5.270833,36.895833,158.125000,0,3500
3,0xf70da97812cb96acdf810712aa562db8dfa3dbef,275100,2025-04-01 00:00:11+00:00,2025-05-03 17:46:35+00:00,368.0,424,0,8.653061,60.571429,259.591837,0,0
4,0x264bd8291fae1d75db2c5f573b07faa6715997b5,270429,2025-04-01 00:00:11+00:00,2025-05-03 17:46:35+00:00,3346.0,214,0,4.458333,31.208333,133.750000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8893,0x15fba39460a78c168f7acb701ecf43ae19cf458e,37,2025-04-03 03:21:35+00:00,2025-05-03 15:39:11+00:00,0.0,1,0,1.000000,7.000000,30.000000,0,0
8894,0x7672c5fa51ea088f5904d66b9f816dcf7ef47cb8,37,2025-04-20 06:14:47+00:00,2025-05-03 16:27:47+00:00,0.0,1,0,1.000000,7.000000,30.000000,0,0
8895,0x97d989470a07932579ebbbdf569f130c5f10ec89,37,2025-04-01 02:42:47+00:00,2025-05-02 23:45:11+00:00,0.0,1,1,1.000000,7.000000,30.000000,0,3500
8896,0x0d6fe0bd1d4a27f952ff0f54e849fd7b8678a36f,37,2025-04-06 17:24:11+00:00,2025-05-03 17:28:47+00:00,0.0,1,0,1.000000,7.000000,30.000000,0,0


In [8]:
df_final.to_csv('/Users/bahareh/Desktop/My_Job/BlochChain/Git/ml-in-crypto/04_Wallet_Canvas/df_final.csv')

# Part 3: Line Charts

## Query for number of tx and balance per month

WITH txs AS (
  SELECT
    from_address,
    to_address,
    value / 1e18 AS eth_value,
    block_timestamp,
    FORMAT_TIMESTAMP('%Y-%m', block_timestamp) AS month
  FROM `bigquery-public-data.crypto_ethereum.transactions`
  WHERE
    block_timestamp BETWEEN '2025-04-01' AND '2025-05-01'
),

eoas AS (
  SELECT DISTINCT address
  FROM (
    SELECT from_address AS address FROM txs
    UNION DISTINCT
    SELECT to_address AS address FROM txs
  )
  WHERE address NOT IN (
    SELECT address FROM `bigquery-public-data.crypto_ethereum.contracts`
  )
),

flows AS (
  SELECT
    month,
    address,
    SUM(received) AS received_eth,
    SUM(sent) AS sent_eth,
    COUNTIF(sent > 0) + COUNTIF(received > 0) AS tx_count
  FROM (
    SELECT
      month,
      to_address AS address,
      eth_value AS received,
      0 AS sent
    FROM txs
    UNION ALL
    SELECT
      month,
      from_address AS address,
      0 AS received,
      eth_value AS sent
    FROM txs
  )
  WHERE address IN (SELECT address FROM eoas)
  GROUP BY month, address
),

wallet_balances AS (
  SELECT
    month,
    address,
    SUM(received_eth - sent_eth) OVER (
      PARTITION BY address
      ORDER BY month
      ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) AS cumulative_balance_eth,
    tx_count
  FROM flows
)

SELECT
  month,
  address AS wallet_address,
  ROUND(cumulative_balance_eth, 6) AS eth_balance,
  tx_count
FROM wallet_balances
ORDER BY wallet_address, month


In [2]:
df3 = pd.read_csv('/Users/bahareh/Desktop/Data/part 3 - 1.csv')

df3.head()

,month,wallet_address,eth_balance,tx_count
0,2024-04,0x0000000000000000000000000000000000000000,1.619556,196
1,2024-05,0x0000000000000000000000000000000000000000,2.686039,44
2,2024-06,0x0000000000000000000000000000000000000000,2.998383,31
3,2024-07,0x0000000000000000000000000000000000000000,6.298392,85
4,2024-08,0x0000000000000000000000000000000000000000,11.651262,273


In [9]:
filtered_df3 = df3[df3['wallet_address'].isin(df_final['wallet_address'])]

filtered_df3.head()

,month,wallet_address,eth_balance,tx_count
392,2024-04,0x00000000000030e5959659622cb7eb50aa20ee52,2.108130,674
393,2024-05,0x00000000000030e5959659622cb7eb50aa20ee52,4.166629,1963
394,2024-06,0x00000000000030e5959659622cb7eb50aa20ee52,7.984797,2747
395,2024-07,0x00000000000030e5959659622cb7eb50aa20ee52,9.946088,2958
396,2024-08,0x00000000000030e5959659622cb7eb50aa20ee52,10.768526,2948


In [10]:
filtered_df3.to_csv('/Users/bahareh/Desktop/My_Job/BlochChain/Git/ml-in-crypto/04_Wallet_Canvas/part3_data.csv')

In [11]:


filtered_df3[filtered_df3['wallet_address']=='0x28c6c06298d514db089934071355e5743bf21d60']

,month,wallet_address,eth_balance,tx_count


In [12]:
df_final[df_final['wallet_address']=='0x28c6c06298d514db089934071355e5743bf21d60']

,wallet_address,tx_count,first_tx,last_tx,eth,total_tx,avg_tx_value,tx_per_day,tx_per_week,tx_per_month,avg_time_gap_days,avg_value_usd
5,0x28c6c06298d514db089934071355e5743bf21d60,267666,2025-04-01 00:00:59+00:00,2025-05-03 17:46:35+00:00,47069.0,472,26,9.632653,67.428571,288.979592,0,91000


In [13]:
print('0x28c6...'.lower() in df3['wallet_address'].str.lower().values)

False


# Part 4:  Tables

## Query for the most intractions between EOA to EOA
WITH contracts AS (
  SELECT LOWER(address) AS contract_address
  FROM `bigquery-public-data.crypto_ethereum.contracts`
),

txs AS (
  SELECT
    LOWER(from_address) AS sender,
    LOWER(to_address) AS receiver
  FROM `bigquery-public-data.crypto_ethereum.transactions`
  WHERE block_timestamp BETWEEN '2024-04-01' AND '2024-04-30'
),

eoa_interactions AS (
  SELECT
    sender AS wallet,
    receiver AS counterparty
  FROM txs
  WHERE sender NOT IN (SELECT contract_address FROM contracts)
    AND receiver NOT IN (SELECT contract_address FROM contracts)
),

ranked_interactions AS (
  SELECT
    wallet,
    counterparty,
    COUNT(*) AS tx_count,
    ROW_NUMBER() OVER (PARTITION BY wallet ORDER BY COUNT(*) DESC) AS rnk
  FROM eoa_interactions
  GROUP BY wallet, counterparty
)

SELECT *
FROM ranked_interactions
WHERE rnk <= 10
ORDER BY wallet, tx_count DESC

In [14]:
df4_a = pd.read_csv('/Users/bahareh/Desktop/Data/part4_a.csv')

df4_a.head()

,wallet,counterparty,tx_count,rnk
0,0x0000000000000383c84db8f4c86a095abcc206bf,0x0000000000000383c84db8f4c86a095abcc206bf,1,3
1,0x0000000000000383c84db8f4c86a095abcc206bf,0xc3b90066d445a2f816055008b36db7e499999999,1,4
2,0x0000000000000383c84db8f4c86a095abcc206bf,0xaf8d6f3c63d5e1062ec41ba88136ee9e20ad5e5f,1,2
3,0x0000000000000383c84db8f4c86a095abcc206bf,0x71bb35f791d7d86f3669674446d80e8ec5b2508f,1,1
4,0x00000000000007736e2f9aa5630b8c812e1f3fc9,0xfcdd6d25ae394746e4c74610aee215aaa4fd2e48,11,1


In [43]:
df4_a = df4_a.rename(columns={'wallet': 'wallet_address'})


df4_a = df4_a[df4_a['wallet_address'].isin(df_final['wallet_address'])]
df4_a.head()

,wallet_address,counterparty,tx_count,rnk
92,0x0000000000ea39366718bfc07b48016b2c12f939,0xff8be2f8d0ea681d8a89e75bd454e83fb931b8f2,1,1
382,0x000000ffa51e8fd8dc15c5b77990234810a640e8,0x000000422a649825bda802b2c212c46127bd96fd,7,1
383,0x000000ffa51e8fd8dc15c5b77990234810a640e8,0x9a05fe00171e6382e4a0098c318f462c53e3d075,4,2
384,0x000000ffa51e8fd8dc15c5b77990234810a640e8,0x2337568ace7bce6d03c61b01c4afb5130c2c999a,1,3
844,0x0000db5c8b030ae20308ac975898e09741e70000,0x0000db5c8b030ae20308ac975898e09741e70000,10,1


## Query for the most used smart contracts EOA → Contract

WITH contracts AS (
  SELECT LOWER(address) AS contract_address
  FROM `bigquery-public-data.crypto_ethereum.contracts`
),

txs AS (
  SELECT
    LOWER(from_address) AS sender,
    LOWER(to_address) AS receiver
  FROM `bigquery-public-data.crypto_ethereum.transactions`
  WHERE block_timestamp BETWEEN '2024-04-01' AND '2024-04-30'
),

eoa_to_contract AS (
  SELECT
    sender AS wallet,
    receiver AS contract
  FROM txs
  WHERE sender NOT IN (SELECT contract_address FROM contracts)
    AND receiver IN (SELECT contract_address FROM contracts)
),

ranked_contracts AS (
  SELECT
    wallet,
    contract,
    COUNT(*) AS tx_count,
    ROW_NUMBER() OVER (PARTITION BY wallet ORDER BY COUNT(*) DESC) AS rnk
  FROM eoa_to_contract
  GROUP BY wallet, contract
)

SELECT *
FROM ranked_contracts
WHERE rnk <= 10
ORDER BY wallet, tx_count DESC

In [15]:
df4_b = pd.read_csv('/Users/bahareh/Desktop/Data/part4_b.csv')

df4_b.head()

,wallet,contract,tx_count,rnk
0,0x0000000000000383c84db8f4c86a095abcc206bf,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,2,1
1,0x0000000000000383c84db8f4c86a095abcc206bf,0x00000000005bbb0ef59571e58418f9a4357b68a0,2,2
2,0x0000000000000383c84db8f4c86a095abcc206bf,0xd55210bb6898c021a19de1f58d27b71f095921ee,1,4
3,0x0000000000000383c84db8f4c86a095abcc206bf,0xcc8e00a444e7d4262fcdc2151fe866741ce76986,1,3
4,0x0000000000000383c84db8f4c86a095abcc206bf,0x5f683665ca87dbc3d1358913da80e3c71c328fb0,1,5


In [16]:
df4_b['contract'].nunique()

8441

In [19]:
df4_b['contract'].value_counts().head()

0xdac17f958d2ee523a2206206994597c13d831ec7    10060
0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad     4992
0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48     4217
0x6774bcbd5cecef1336b5300fb5186a12ddd8b367     3519
0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2     1616
Name: contract, dtype: int64

In [36]:
contract_names = {
    '0xdac17f958d2ee523a2206206994597c13d831ec7': 'Tether (USDT)',
    '0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad': 'Uniswap Router',
    '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48': 'USD Coin (USDC)',
    '0x6774bcbd5cecef1336b5300fb5186a12ddd8b367': 'OpenSea Seaport',
    '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2': 'Wrapped Ether (WETH)'}



contract_names.update({
    '0x1522900b6dafac587d499a862861c0869be6e428': 'Bitfinex Wallet',
    '0xffa397285ce46fb78c588a9e993286aac68c37cd': 'Safe Multisig',
    '0x6f1cdbbb4d53d226cf4b917bf768b94acbab6168': 'LayerZero Endpoint',
    '0xc21e4ebd1d92036cb467b53fe3258f219d909eb9': 'Zora Marketplace',
    '0x5079fc00f00f30000e0c8c083801cfde000008b6': 'Unknown Contract #1',
    '0x93ffb15d1fa91e0c320d058f00ee97f9e3c50096': 'Unknown Contract #2',
    '0x3328f7f4a1d1c57c35df56bbf0c9dcafca309c49': 'Binance Bridge',
    '0x881d40237659c251811cec9c364ef91dc08d300c': 'MetaMask Swap Router',
    '0xdef1c0ded9bec7f1a1670819833240f027b25eff': '0x Exchange Proxy',
    '0x1111111254eeb25477b68fb85ed929f73a960582': '1inch Aggregation Router v3',
    '0x7a250d5630b4cf539739df2c5dacb4c659f2488d': 'Uniswap V2 Router',
    '0x1d50d75933b7b7c8ad94dbfb748b5756e3889c24': 'Unknown Contract #3',
    '0x80a64c6d7f12c47b7c66c5b4e20e72bc1fcd5d9e': 'Unknown Contract #4',
    '0x63756a3c3bf677baab9d9e06457402ed05be8570': 'Blur Marketplace',
    '0x64545160d28fd0e309277c02d6d73b3923cc4bfa': 'Unknown Contract #5',
    '0xa9d1e08c7793af67e9d92fe308d5697fb81d3e43': 'Unknown Contract #6',
    '0xf3de3c0d654fda23dad170f0f320a92172509127': 'Unknown Contract #7',
    '0x111111125421ca6dc452d289314280a0f8842a65': '1inch Aggregation Router v4',
    '0x77f0de655885dcf6b6942ea5b3de171dfd3f5da9': 'Unknown Contract #8',
    '0xee2a0343e825b2e5981851299787a679ce08216d': 'Unknown Contract #9',
})

In [37]:
df4_b['contract_name'] = df4_b['contract'].str.lower().map(contract_names)
df4_b

,wallet,contract,tx_count,rnk,contract_name
0,0x0000000000000383c84db8f4c86a095abcc206bf,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,2,1,Uniswap V2 Router
1,0x0000000000000383c84db8f4c86a095abcc206bf,0x00000000005bbb0ef59571e58418f9a4357b68a0,2,2,NaN
2,0x0000000000000383c84db8f4c86a095abcc206bf,0xd55210bb6898c021a19de1f58d27b71f095921ee,1,4,NaN
3,0x0000000000000383c84db8f4c86a095abcc206bf,0xcc8e00a444e7d4262fcdc2151fe866741ce76986,1,3,NaN
4,0x0000000000000383c84db8f4c86a095abcc206bf,0x5f683665ca87dbc3d1358913da80e3c71c328fb0,1,5,NaN
...,...,...,...,...,...
96119,0x02b8c47644efa9d14637eee0cd7c6b28fe9ef4cd,0xd0c155595929fd6be034c3848c00daebc6d330f6,1,2,NaN
96120,0x02b8c47644efa9d14637eee0cd7c6b28fe9ef4cd,0x36e66fbbce51e4cd5bd3c62b637eb411b18949d4,1,1,NaN
96121,0x02b8cd29aac76b6da7f4fb46691259300ee059d3,0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0,1,1,NaN
96122,0x02b8cdcec7f755e4b91c64660e75a377500e2037,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,3,1,Uniswap Router


In [41]:
df4_b = df4_b.rename(columns={'wallet': 'wallet_address'})


df4_b = df4_b[df4_b['wallet_address'].isin(df_final['wallet_address'])]
df4_b.head()

,wallet_address,contract,tx_count,rnk,contract_name
7,0x00000000000030e5959659622cb7eb50aa20ee52,0x63756a3c3bf677baab9d9e06457402ed05be8570,617,1,Blur Marketplace
32,0x00000000000124d994209fbb955e0217b5c2eca1,0x63756a3c3bf677baab9d9e06457402ed05be8570,1753,1,Blur Marketplace
174,0x0000000000ea39366718bfc07b48016b2c12f939,0x0000000000fec33c7a7e8330012782cea32ffd1c,28,1,NaN
175,0x0000000000ea39366718bfc07b48016b2c12f939,0x7547042ec75e410f1cea4da0e37676241d04ceac,2,2,NaN
176,0x0000000000ea39366718bfc07b48016b2c12f939,0x4e59b44847b379578588920ca78fbf26c0b4956c,1,3,NaN


In [42]:
df4_b.to_csv('/Users/bahareh/Desktop/My_Job/BlochChain/Git/ml-in-crypto/04_Wallet_Canvas/df4_b_data.csv')